##Git and GitHub
Define access credentials and repo details.

The following environment variables (or secret keys) are assumed to be configured:
* GITHUB_USERNAME - github username
* GITHUB_USER_EMAIL - email configured for the github user
* GITHUB_TOKEN - github personal access token with relevant privileges (`repo` is enough)


In [1]:
from google.colab import userdata

github_user_email = userdata.get('GITHUB_USER_EMAIL')

In [3]:
token = userdata.get('GITHUB_TOKEN')
username = userdata.get('GITHUB_USERNAME')
repo = "mini-food-pic-classifier"

In [17]:
!git config --global user.name {username}
!git config --global user.email {github_user_email}
# !git config --global user.password ""

Get the latest code from the project repo.

Change the repo name and path as you need.

Create new repo in case you want to use a new repo.

In [4]:
!git clone https://{token}@github.com/{username}/{repo}

Cloning into 'mini-food-pic-classifier'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), done.


Change the working path to be inside the repo folder from this point and on.

In [5]:
%cd {repo}

/content/mini-food-pic-classifier


In [6]:
# project_name = "mini-food-pic-classifier"
project_path = "/content/" + repo

Currently assume the repo does not have the most updated version of this notebook, so copy it from the google drive:
* Connect to the drive
* Copy the latest notebook from the drive to its place in the local git repo

In [7]:
import os
load_notebooks_path = os.path.join(project_path, "notebooks")
if not os.path.exists(load_notebooks_path):
  os.mkdir(load_notebooks_path)

In [37]:
notebook_path = "/content/drive/MyDrive/Colab Notebooks/youtube/Learn PyTorch for deep learning in a day/05_pytorch_going_modular/pytorch_going_modular_cell_mode.ipynb"

In [8]:
!cp  {notebook_path} ./notebooks/

In [9]:
!git status

On branch main
Your branch is up to date with 'origin/main'.

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	notebooks/

nothing added to commit but untracked files present (use "git add" to track)


In [10]:
!git checkout -b cell_mode_get_data

Switched to a new branch 'cell_mode_get_data'


In [19]:
!git add notebooks
!git commit -a -m "Get data in cell mode"

[cell_mode_get_data 06fef79] Get data in cell mode
 1 file changed, 1 insertion(+)
 create mode 100644 notebooks/pytorch_going_modular_cell_mode.ipynb


In [20]:
!git status

On branch cell_mode_get_data
nothing to commit, working tree clean


In [22]:
!git push origin main

Everything up-to-date


##0. Running a notebook in cell mode

This notebook is intended to run normally, one cell at a time.

## 1. Get data
We're going to start by downloading the `pizza_steak_sushi` datast with images of pizza, steak and sushi.

In [29]:
import os
import zipfile

from pathlib import Path

import requests

# Setup path to data folder
data_path = Path(os.path.join(project_path, "data/"))
image_path = data_path / "pizza_steak_sushi"

# If the image folder doesn't exist, download it and prepare it...
if image_path.is_dir():
    print(f"{image_path} directory exists.")
else:
    print(f"Did not find {image_path} directory, creating one...")
    image_path.mkdir(parents=True, exist_ok=True)

# Download pizza, steak, sushi data
with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading pizza, steak, sushi data...")
    f.write(request.content)

# Unzip pizza, steak, sushi data
with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    print("Unzipping pizza, steak, sushi data...")
    zip_ref.extractall(image_path)

# Remove zip file
os.remove(data_path / "pizza_steak_sushi.zip")

Did not find /content/mini-food-pic-classifier/data/pizza_steak_sushi directory, creating one...
Unzipping pizza, steak, sushi data...


In [30]:
# Setup train and testing paths
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(PosixPath('/content/mini-food-pic-classifier/data/pizza_steak_sushi/train'),
 PosixPath('/content/mini-food-pic-classifier/data/pizza_steak_sushi/test'))

##2. Create Datasets and DataLoaders
Let's turn our data into PyTorch `Dataset`'s and `DataLoader`'s and find out a few useful attributes from them such as `classes` and their lengths.

In [31]:
from torchvision import datasets, transforms

# Create simple transform
data_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Use ImageFolder to create dataset(s)
train_data = datasets.ImageFolder(root=train_dir, # target folder of images
                                  transform=data_transform, # transforms to perform on data (images)
                                  target_transform=None) # transforms to perform on labels (if necessary)

test_data = datasets.ImageFolder(root=test_dir,
                                 transform=data_transform)

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

Train data:
Dataset ImageFolder
    Number of datapoints: 225
    Root location: /content/mini-food-pic-classifier/data/pizza_steak_sushi/train
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Test data:
Dataset ImageFolder
    Number of datapoints: 75
    Root location: /content/mini-food-pic-classifier/data/pizza_steak_sushi/test
    StandardTransform
Transform: Compose(
               Resize(size=(64, 64), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )


In [32]:
# Get class names as a list
class_names = train_data.classes
class_names

['pizza', 'steak', 'sushi']

In [33]:
# Can also get class names as a dict
class_dict = train_data.class_to_idx
class_dict

{'pizza': 0, 'steak': 1, 'sushi': 2}

In [34]:
# Check the lengths
len(train_data), len(test_data)

(225, 75)

In [35]:
# Turn train and test Datasets into DataLoaders
from torch.utils.data import DataLoader

train_dataloader = DataLoader(dataset=train_data,
                              batch_size=1, # how many samples per batch?
                              num_workers=1, # how many subprocesses to use for data loading? (higher = more)
                              shuffle=True) # shuffle the data?

test_dataloader = DataLoader(dataset=test_data,
                             batch_size=1,
                             num_workers=1,
                             shuffle=False) # don't usually need to shuffle testing data

train_dataloader, test_dataloader

(<torch.utils.data.dataloader.DataLoader at 0x7ba6fb41b190>,
 <torch.utils.data.dataloader.DataLoader at 0x7ba6fb41b100>)

In [36]:
# Check out single image size/shape
img, label = next(iter(train_dataloader))

# Batch size will now be 1, try changing the batch_size parameter above and see what happens
print(f"Image shape: {img.shape} -> [batch_size, color_channels, height, width]")
print(f"Label shape: {label.shape}")

Image shape: torch.Size([1, 3, 64, 64]) -> [batch_size, color_channels, height, width]
Label shape: torch.Size([1])


###Update Git

In [38]:
!cp {notebook_path} ./notebooks/
!git checkout -b create_datasets_and_dataloaders
!git add notebooks/pytorch_going_modular_cell_mode.ipynb
!git commit -a -m "Create Datasets and DataLoaders"
!git push origin main

cp: cannot stat '/content/drive/MyDrive/Colab': No such file or directory
cp: cannot stat 'Notebooks/youtube/Learn': No such file or directory
cp: cannot stat 'PyTorch': No such file or directory
cp: cannot stat 'for': No such file or directory
cp: cannot stat 'deep': No such file or directory
cp: cannot stat 'learning': No such file or directory
cp: cannot stat 'in': No such file or directory
cp: cannot stat 'a': No such file or directory
cp: cannot stat 'day/05_pytorch_going_modular/pytorch_going_modular_cell_mode.ipynb': No such file or directory
Switched to a new branch 'create_datasets_and_dataloaders'
On branch create_datasets_and_dataloaders
Untracked files:
  (use "git add <file>..." to include in what will be committed)
	data/

nothing added to commit but untracked files present (use "git add" to track)
Everything up-to-date
